<a href="https://colab.research.google.com/github/tomocraft/RinGod-Colab/blob/main/RinGod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まず、３セル目のコードの app.config.update() の中身を適宜変えてください。メールアドレス認証はこの中の設定がないと動きません。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

rinGod_folder = "/content/drive/MyDrive/RinGod"

rinGodF_folder = "/content/drive/MyDrive/RinGod/RinGod-FrontEnd"

os.makedirs(rinGod_folder, exist_ok=True)

%cd {rinGod_folder}

!git clone https://github.com/Tomocraft/RinGod-FrontEnd.git
!git clone https://github.com/Tomocraft/RinGod-Data.git

%cd {rinGodF_folder}

!git stash
!git pull
!git stash pop

In [ ]:
!pip install flask pyngrok pyjwt flask_mail flask_bcrypt itsdangerous flask_socketio

import subprocess
lt_process = subprocess.Popen(
    ["npx", "--yes", "localtunnel", "--port", "5000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
import time
time.sleep(2)
output = lt_process.stdout.readline()
print(output.decode('utf-8'))

import requests

def get_colab_global_ip():
    try:
        ip = requests.get("https://api.ipify.org").text
        return ip
    except Exception as e:
        return f"IP取得失敗: {e}"

print("LocalTunnel Password: ", get_colab_global_ip())

from flask import Flask, request, jsonify, send_from_directory, url_for
from flask_socketio import SocketIO, send, disconnect
import jwt
from datetime import datetime, timedelta, timezone
# from pyngrok import ngrok
from flask_mail import Mail, Message
from flask_bcrypt import Bcrypt
from itsdangerous import URLSafeTimedSerializer, SignatureExpired, BadSignature
import os
import json


TARGET = "/content/drive/MyDrive/RinGod/RinGod-FrontEnd"

USERS_FILE = "/content/drive/MyDrive/RinGod/RinGod-Data/users.json"

BOARDS_FILE = "/content/drive/MyDrive/RinGod/RinGod-Data/boards.json"

BOARDS_FOLDER = "/content/drive/MyDrive/RinGod/RinGod-Data/boards"

def load_users():
    if not os.path.exists(USERS_FILE):
        return {}
    with open(USERS_FILE, "r", encoding="utf-8") as f:
        try:
            users = json.load(f)
        except json.JSONDecodeError:
            return {}
    verified_users = {email: data for email, data in users.items() if data.get("verified", False)}
    return verified_users

def save_users(users):
    with open(USERS_FILE, "w", encoding="utf-8") as f:
        json.dump(users, f, ensure_ascii=False, indent=4)

users = load_users()

JST = timezone(timedelta(hours=+9), 'JST')

app = Flask(__name__)

# ngrok.set_auth_token("") # ここにngrokのtokenを貼り付ける

app.config.update(
    SECRET_KEY="a93f1c5c7d1e8bca2f8d7b4e23b2c49d1e4e718fc0aeb2e3e2e528bdbe821a5a", # ここには適当に秘密鍵を作って貼り付けてください
    MAIL_SERVER="smtp.gmail.com",
    MAIL_PORT=587,
    MAIL_USE_TLS=True,
    MAIL_USERNAME="", # ここにはGmailのメールアドレスを書く
    MAIL_PASSWORD="", # Gmailのアプリパスワードを書く
)

mail = Mail(app)
bcrypt = Bcrypt(app)
serializer = URLSafeTimedSerializer(app.config["SECRET_KEY"])

sockets = {}

socketio = SocketIO(app)

@socketio.on("connect")
def handle_connect(auth):
    username = "";
    if request.args.get("email") and request.args.get("username"):
        email = request.args.get("email")
        username = request.args.get("username")
        sockets[username] = request.sid
        return True
    token = auth.get("token") if auth else None
    if not token:
        return False
    try:
        if token.startswith("Bearer "):
            token = token[7:]
        data = jwt.decode(token, app.config["SECRET_KEY"], algorithms=["HS256"])
        return socketio.emit("message", {"username": data["username"]}, to=request.sid)
    except jwt.ExpiredSignatureError:
        return socketio.emit("message", {"message": "トークンの有効期限が切れています"}, to=request.sid)
    except jwt.InvalidTokenError:
        return socketio.emit("message", {"message": "無効なトークンです"}, to=request.sid)
    print(username + " がソケットに接続")

@app.route("/")
def home():
  return send_from_directory(TARGET, "index.html")

@app.route("/<path:filename>")
def showFile(filename):
  return send_from_directory(TARGET, filename)

@app.route("/backend/login", methods=["POST"])
def login():
    data = request.form
    identifier = data.get("username", "").strip()
    password = data.get("password", "")
    user = users.get(identifier)
    if not user:
        for email, info in users.items():
            if info["username"].lower() == identifier.lower():
                user = info
                break
    if not user:
        return jsonify({"message": "ユーザー名・メールアドレスまたはパスワードが間違っています。"}), 401
    if not bcrypt.check_password_hash(user["password"], password):
        return jsonify({"message": "ユーザー名・メールアドレスまたはパスワードが間違っています。"}), 401
    payload = {
        "username": user["username"],
        "exp": datetime.utcnow() + timedelta(days=7)
    }
    token = jwt.encode(payload, app.config["SECRET_KEY"], algorithm="HS256")
    return jsonify({"token": token})

@app.route("/backend/profile")
def profile():
    token = request.headers.get("Authorization")
    if not token:
        return jsonify({"message": "トークンが必要です"}), 401
    try:
        if token.startswith("Bearer "):
            token = token[7:]
        data = jwt.decode(token, app.config["SECRET_KEY"], algorithms=["HS256"])
        return jsonify({"username": data["username"]})
    except jwt.ExpiredSignatureError:
        return jsonify({"message": "トークンの有効期限が切れています"}), 401
    except jwt.InvalidTokenError:
        return jsonify({"message": "無効なトークンです"}), 401

@app.route("/backend/signup", methods=["POST"])
def signup():
    data = request.form
    username = data.get("username", "").strip()
    email = data.get("email", "").strip()
    password = data.get("password", "")
    if not username or not email or not password:
        return jsonify({"message": "すべての項目を入力してください。"}), 400
    if email in users:
        return jsonify({"message": "このメールアドレスはすでに登録されています。"}), 400
    for u in users.values():
        if u["username"].lower() == username.lower():
            return jsonify({"message": "このユーザー名はすでに使用されています。"}), 400
    hashed_pw = bcrypt.generate_password_hash(password).decode("utf-8")
    users[email] = {"username": username, "password": hashed_pw, "verified": False}
    token = serializer.dumps(email, salt="email-confirm")
    confirm_url = url_for("confirm_email", token=token, _external=True)
    msg = Message("メールアドレスの確認", sender=app.config["MAIL_USERNAME"], recipients=[email])
    msg.body = f"""{username} 様

このたびは当サービスへのご登録ありがとうございます。

以下のリンクをクリックして、メールアドレスの確認をお願いいたします。

{confirm_url}

※リンクの有効期限は発行から1時間です。期限を過ぎた場合は再度アカウント登録をお願いいたします。

身に覚えのない場合は、このメールを無視してください。
    """
    mail.send(msg)
    save_users(users)
    return jsonify({"message": "確認メールを送信しました。受信ボックスを確認してください。"})

@app.route("/backend/bbs/getmsg", methods=["POST"])
def getmsg():
    data = request.get_json()
    boardId = data.get("boardId")
    with open(BOARDS_FOLDER + "/" + str(boardId) + ".json", "r", encoding="utf-8") as f:
        try:
            return jsonify(json.load(f))
        except FileNotFoundError:
            with open(BOARDS_FOLDER + "/" + str(boardId) + ".json", "w", encoding="utf-8") as f:
                f.write("[]")
                return jsonify([])
        except json.JSONDecodeError:
            return {}, 401

@app.route("/backend/bbs/sendmsg", methods=["POST"])
def sendmsg():
    token = request.headers.get("Authorization")
    if not token:
        return jsonify({"message": "トークンが必要です"}), 401
    try:
        if token.startswith("Bearer "):
            token = token[7:]
        userData = jwt.decode(token, app.config["SECRET_KEY"], algorithms=["HS256"])
    except jwt.ExpiredSignatureError:
        return jsonify({"message": "トークンの有効期限が切れています"}), 401
    except jwt.InvalidTokenError:
        return jsonify({"message": "無効なトークンです"}), 401
    data = request.get_json()
    boardId = data.get("boardId")
    message = data.get("message")
    now = datetime.now(JST)
    date_str = now.strftime("%Y/%m/%d %H:%M")
    with open(BOARDS_FILE, "r", encoding="utf-8") as f:
        try:
            ll = []
            for jl in json.load(f):
                if jl["id"] == boardId and userData["username"] not in jl["members"]:
                    jl["members"].append(userData["username"])
                ll.append(jl)
        except json.JSONDecodeError:
            return {}, 401
    with open(BOARDS_FILE, "w", encoding="utf-8") as f:
        json.dump(ll, f, ensure_ascii=False, indent=4)
    with open(BOARDS_FOLDER + "/" + str(boardId) + ".json", "r", encoding="utf-8") as f:
        try:
            d = json.load(f)
        except json.JSONDecodeError:
            return {}, 401
    max_id = max((item["message_id"] for item in d), default=0)
    msg = {
        "message_id": max_id + 1,
        "sentby": {
            "icon": "/icons/i1.png",
            "name": userData["username"]
        },
        "date": date_str,
        "font": "",
        "text": message.replace("\n", "<br>")
    }
    d.append(msg)
    with open(BOARDS_FOLDER + "/" + str(boardId) + ".json", "w", encoding="utf-8") as f:
        json.dump(d, f, ensure_ascii=False, indent=4)
    socketio.emit("msg", msg)
    return jsonify({"username": userData["username"]})

@app.route("/backend/bbs/list")
def getList():
    with open(BOARDS_FILE, "r", encoding="utf-8") as f:
        try:
            return jsonify((json.load(f))[:15])
        except json.JSONDecodeError:
            return {}, 401

@app.route("/backend/bbs/load", methods=["POST"])
def loadMore():
    data = request.get_json()
    loaded = data.get("loaded")
    with open(BOARDS_FILE, "r", encoding="utf-8") as f:
        try:
            return jsonify((json.load(f))[loaded:15])
        except json.JSONDecodeError:
            return {}, 401

@app.route("/backend/bbs/post", methods=["POST"])
def postBoard():
    token = request.headers.get("Authorization")
    if not token:
        return jsonify({"message": "トークンが必要です"}), 401
    try:
        if token.startswith("Bearer "):
            token = token[7:]
        userData = jwt.decode(token, app.config["SECRET_KEY"], algorithms=["HS256"])
    except jwt.ExpiredSignatureError:
        return jsonify({"message": "トークンの有効期限が切れています"}), 401
    except jwt.InvalidTokenError:
        return jsonify({"message": "無効なトークンです"}), 401
    data = request.get_json()
    title = data.get("title")
    message = data.get("message")
    now = datetime.now(JST)
    date_str = now.strftime("%Y/%m/%d")
    with open(BOARDS_FILE, "r", encoding="utf-8") as f:
        d = json.load(f)
    max_id = max((item["id"] for item in d), default=0)
    post = {
        "id": max_id + 1,
        "title": title,
        "madeby": {
            "icon": "/icons/i1.png",
            "name": userData["username"],
            "date": date_str
        },
        "members": [userData["username"]],
        "message": {
            "font": "",
            "message": message.replace("\n", "<br>")
        }
    }
    with open(BOARDS_FOLDER + "/" + str(max_id + 1) + ".json", "w", encoding="utf-8") as f:
        f.write("[]")
    with open(BOARDS_FILE, "w", encoding="utf-8") as f:
        json.dump([post] + d, f, ensure_ascii=False, indent=4)
    socketio.emit("update", post)
    return jsonify(post)

@app.route("/confirm/<token>")
def confirm_email(token):
    try:
        email = serializer.loads(token, salt="email-confirm", max_age=3600)
    except SignatureExpired:
        return "リンクの有効期限が切れています。", 400
    except BadSignature:
        return "無効なトークンです。", 400
    user = users.get(email)
    if not user:
        return "ユーザーが存在しません。", 404
    user["verified"] = True
    save_users(users)
    socketio.emit("message", "verified", to=sockets[user["username"]])
    return user['username'] + " さん、メールアドレスが確認されました"

# public_url = ngrok.connect(5000)
# print("Access URL:", public_url)

app.run(port=5000)